1.	We defined the Gini index value for a node in a classification tree as
$$2 p (1-p)$$
Where p is the estimated probability of either of the two classes.  For example, if a node has 30 training instances of class A, and 50 training instances of class B, then the estimated probability of class A is $$30/(30 + 50)$$  What is the Gini index value for this node?



2.	Create a new Python file.  Fill in the code below to create a function that gives the Gini value for a node in a binary classification tree given values for the number of instances of each class.  class_counts is a list of length two.   

In [2]:
def gini(class counts):
    ''' return the Gini value for a node in a binary classif. tree '''
    # your code here (don't forget the return statement)
    pass

SyntaxError: invalid syntax (<ipython-input-2-ba7a1272172c>, line 1)

3.	Test your function.  What is gini([30, 50])? gini([10, 10])?  What is gini([20, 0])?  What is gini([100, 0])? 

4.	Add the following code at the top of your file to read and preprocess the data.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/grbruns/cst383/master/heart.csv")
df['output'] = df['output'] - 1

df = df[['age', 'maxhr', 'restbp', 'output']]
sns.scatterplot(x='age', y='maxhr', hue='output', data=df)


5.	Run the code and look at the plot.  If we were going to build a classification tree, and split first on 'age', what do you think a good age value to split on would be?

6.	Compute the Gini index for df as a whole.  For this you just need the number of rows with output = 0 and the number of rows with output = 1.

7.	Now consider a split on age < 50.  Write code to compute the Gini index for the case of of age < 50 and the Gini index for the case of age >= 50.  For the case of age < 50, get the rows of df where age < 50, then count the number of rows with output = 0 and output = 1.  

8.	Now compute the overall Gini index value for the split on age < 50.  First you need to compute the fraction of nodes associated with age < 50 (call it fraction_lo) and the fraction of nodes associated with age >= 50 (call it fraction_hi).  Then get the Gini value for the split like this (in pseudo code):  
```gini_split = gini_lo * fraction_lo + gini_hi * fraction_hi```  
  
The split is useful if the Gini value for the split is lower than the GIni value for the root.

9.	Is a split on age < 40 better than a split on age < 50?

10.	Compute the Gini value for all age splits where age ranges from 20 to 80.  Then plot the Gini split value for all the ages (age on x axis, Gini value on y axis).  What is the best age value for a split on age?

11.	If you still have time, do the same thing for features 'maxhr' and 'restbp'.  What is the best feature and best split?